In [1]:
import datetime
import json
import os
import sqlite3

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots


# Load Demographics and Dataset_Part-Patient_id bindings


In [2]:
BASE_DATASETS_PATH = "D:/Data/Thesis_datasets"
DB_PATH = f"{BASE_DATASETS_PATH}/raw_data.db"
RAW_OBS_PATH = f"{BASE_DATASETS_PATH}/raw_stage/observation_tables"
lactate_var_ids = ("24000524", "24000732", "24000485")

df_demo = pd.read_csv(
    f"{BASE_DATASETS_PATH}/reference_data/general_table.csv",
    dtype={
        "patientid": int,
        "sex": str,
        "age": int,
        "discharge_status": str,
    },
    parse_dates=["admissiontime"],
)
display(df_demo.describe(include="all", datetime_is_numeric=True))

df_pid_part = pd.read_csv(
    f"{RAW_OBS_PATH}/observation_tables_index.csv",
    dtype={
        "patientid": int,
        "part": int,
    },
)
df_pid_part.sort_values(by=["part"], inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Data/Thesis_datasets/reference_data/general_table.csv'

# Move Everything to SQLite
Skip if you already have the DB


In [ ]:
for x in df_pid_part["part"].unique():
    if (x + 1) % 25 == 0:
        print((x + 1) / len(df_pid_part["part"].unique()))
    df_part_X = pd.read_csv(
        f"{RAW_OBS_PATH}/csv/part-{x}.csv",
        dtype={
            "value": str,
            "patientid": int,
            "status": int,
            "stringvalue": str,
            "type": str,
            "value": str,
            "variableid": str,
        },
        parse_dates=["datetime", "entertime"],
    )
    conn = sqlite3.connect(DB_PATH)
    df_part_X.to_sql(name="raw_data", con=conn, if_exists="append", index=False)
    conn.close()

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
print("Creating index on datetime...")
cur.execute("CREATE INDEX index_datetime ON raw_data(datetime)")
print("Creating index on patientid...")
cur.execute("CREATE INDEX index_patient_id ON raw_data(patientid)")
print("Creating index on variableid...")
cur.execute("CREATE INDEX index_variable_id ON raw_data(variableid)")
conn.close()


# Filtering
Skip if you already have the .txt filter files


## Filter Age >= 18 Years Old

No effect, patients inside HiRID are all >= 18 years old


In [42]:
df_demo = df_demo[df_demo["age"] >= 18]
df_demo.reset_index(drop=True, inplace=True)


## Filter Stay >= 1 Day


In [7]:
filtered_pids = list()

conn = sqlite3.connect(DB_PATH)
df_raw = pd.read_sql(
    "SELECT patientid, MAX(datetime) as last_datetime FROM raw_data GROUP BY patientid",
    conn,
    parse_dates=["last_datetime"],
    chunksize=1000,
)
for df in df_raw:
    for tup in df.itertuples():
        # last datetime - admissiontime
        if (
            tup.last_datetime
            - df_demo[df_demo["patientid"] == tup.patientid].iloc[0]["admissiontime"]
        ) >= datetime.timedelta(days=1):
            filtered_pids.append(tup.patientid)
    print(len(filtered_pids))
conn.close()

with open("./results/patients_18+_1d+.txt", "w") as f:
    for x in filtered_pids:
        f.write(f"{x}\n")
print(len(filtered_pids))


511
991
1495
1988
2503
3003
3503
4000
4509
4999
5525
6013
6514
7020
7513
8000
8479
8979
9492
9994
10470
10995
11479
11983
12472
12962
13446
13919
14413
14901
15366
15853
16339
16787
16787


## Filter Lactate_Measurements >= 2

This should be useless as the next filter tests that lactate_measurements >= 2 in the first 2d (CHECK)

So obviously if there are 2+ lactate measurements in the first 2d then there are of course 2+ lactate measurements in general


In [4]:
filtered_pids = list()
previously_filtered = list()
with open("./results/patients_18+_1d+.txt", "r") as f:
    previously_filtered.extend(int(x) for x in f.readlines())

conn = sqlite3.connect(DB_PATH)
lact_vars = "', '".join([str(x) for x in lactate_var_ids])
pids_checks = ", ".join([str(x) for x in previously_filtered])
df_raw = pd.read_sql(
    f"SELECT patientid, COUNT(*) as cnt_lactate_measurements FROM raw_data WHERE (variableid IN ('{lact_vars}')) AND (patientid IN ({pids_checks})) GROUP BY patientid",
    conn,
    chunksize=1000,
)
for df in df_raw:
    for tup in df.itertuples():
        # 2+ lactate measurements
        if tup.cnt_lactate_measurements >= 2:
            filtered_pids.append(tup.patientid)
    print(len(filtered_pids))
conn.close()

with open("./results/patients_18+_1d+_2+lactate_measurements.txt", "w") as f:
    for x in filtered_pids:
        f.write(f"{x}\n")
print(len(filtered_pids))


980
1958
2941
3922
4906
5881
6863
7839
8816
9798
10775
11762
12740
13720
14701
15209
15209


## Filter Lactate_Measurements >= 2 in First 2d of Stay


In [6]:
filtered_pids = list()
previously_filtered = list()
with open("./results/patients_18+_1d+_2+lactate_measurements.txt", "r") as f:
    previously_filtered.extend(int(x) for x in f.readlines())

conn = sqlite3.connect(DB_PATH)
lact_vars = "', '".join([str(x) for x in lactate_var_ids])
for pid in previously_filtered:
    post_2d = df_demo[df_demo["patientid"] == pid].iloc[0][
        "admissiontime"
    ] + datetime.timedelta(days=2)
    df_raw = pd.read_sql(
        f"SELECT patientid, COUNT(*) AS cnt_lactate_measurements FROM raw_data WHERE (variableid IN ('{lact_vars}')) AND (patientid={pid}) AND (datetime<='{post_2d}')",
        conn,
    )
    for tup in df_raw.itertuples():
        # 2+ lactate measurements in first 2d
        if tup.cnt_lactate_measurements >= 2:
            filtered_pids.append(pid)
    if len(filtered_pids) % 1000 == 0:
        print(len(filtered_pids))
conn.close()

with open("./results/patients_18+_1d+_2+lactate_measurements_in_first2d.txt", "w") as f:
    for x in filtered_pids:
        f.write(f"{x}\n")
print(len(filtered_pids))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
15046


# Extract 5 Patients


In [7]:
patientids = list()
with open("./results/patients_18+_1d+_2+lactate_measurements_in_first2d.txt", "r") as f:
    patientids.extend(int(x) for x in f.readlines())
# patientids = patientids[:5]
patientids = [17786, 16503, 22946, 9465, 6586]
# create a folder for each patient
for pid in patientids:
    os.makedirs(f"./results/{pid}", exist_ok=True)
patientids


[17786, 16503, 22946, 9465, 6586]

# Split HiRID in patients' datasets


In [8]:
var_bindings = dict()
with open("./var_id_var_name_bindings.json", "r") as f:
    var_bindings = json.load(f)

for pid in patientids:
    os.makedirs(f"./results/{pid}", exist_ok=True)
    var_ids_checks = "', '".join(var_bindings["dynamic"].keys())
    conn = sqlite3.connect(DB_PATH)
    df_raw = pd.read_sql(
        f"SELECT * FROM raw_data WHERE (patientid={pid}) AND (variableid IN ('{var_ids_checks}')) ORDER BY datetime, entertime",
        conn,
        parse_dates=["datetime", "entertime"],
    )
    df_raw.to_csv(f"./results/{pid}/{pid}_original.csv")
    conn.close()


In [10]:
var_bindings = dict()
with open("./var_id_var_name_bindings.json", "r") as f:
    var_bindings = json.load(f)

static_columns = list(var_bindings["static"].values())
for tup in df_demo[df_demo["patientid"].isin(patientids)].itertuples():
    conn = sqlite3.connect(DB_PATH)
    last_datetime = pd.read_sql(
        f"SELECT MAX(datetime) as last_datetime FROM raw_data WHERE patientid={tup.patientid}",
        conn,
        parse_dates=["last_datetime"],
    ).iloc[0]["last_datetime"]
    df_static_sample = pd.DataFrame(columns=static_columns)
    df_static_sample.loc[0] = [
        getattr(tup, x, None) for x in var_bindings["static"].keys()
    ]
    # compute length of stay
    df_static_sample.at[0, "Length of stay (days)"] = round(
        (last_datetime - df_static_sample.iloc[0]["Admission Time"]).total_seconds()
        / 60
        / 60
        / 24,
        2,
    )
    df_static_sample.to_csv(f"./results/{tup.patientid}/{tup.patientid}_static.csv")

# avoid duplicated columns
dynamic_columns = list(dict.fromkeys(var_bindings["dynamic"].values()).keys())
var_ids_checks = "', '".join(var_bindings["dynamic"].keys())
for pid in patientids:
    conn = sqlite3.connect(DB_PATH)
    df_pid_raw_data = pd.read_sql(
        f"SELECT * FROM raw_data WHERE (patientid={pid}) AND (variableid IN ('{var_ids_checks}')) ORDER BY datetime, entertime",
        conn,
        parse_dates=["datetime", "entertime"],
    )
    conn.close()
    pid_admission_time = df_demo[df_demo["patientid"] == pid]["admissiontime"].iloc[0]
    tmp_dict = dict()
    for tup in df_pid_raw_data.itertuples():
        row_key = round((tup.datetime - pid_admission_time).total_seconds() / 60, 2)
        if row_key not in tmp_dict:
            tmp_dict[row_key] = dict.fromkeys(dynamic_columns)
            tmp_dict[row_key][var_bindings["dynamic"]["datetime"]] = tup.datetime
            # entertime could be different for different values of the same record, maybe we should remove it
            # i.e. some values have the same datetime (observation) but different entertime (insertion into db)
            tmp_dict[row_key][var_bindings["dynamic"]["entertime"]] = tup.entertime

        if var_bindings["dynamic"][tup.variableid] == "Lactate":
            # check lactate variables priority (use arterial, if not available use venous)
            if tup.variableid != "24000524":
                # not arterial lactate, check if there's some lactate value already present
                if (
                    tmp_dict[row_key][var_bindings["dynamic"][tup.variableid]]
                    is not None
                ):
                    # lactate value already present, skip since it's probably an arterial one
                    continue

        tmp_dict[row_key][var_bindings["dynamic"][tup.variableid]] = tup.value

        # create new vars for bp
        if var_bindings["dynamic"][tup.variableid] in (
            "Invasive Diastolic Arterial Blood Pressure",
            "Non-Invasive Diastolic Arterial Blood Pressure",
            "Invasive Mean Arterial Blood Pressure",
            "Non-Invasive Mean Arterial Blood Pressure",
            "Invasive Systolic Arterial Blood Pressure",
            "Non-Invasive Systolic Arterial Blood Pressure",
        ):
            var_id = ""
            if var_bindings["dynamic"][tup.variableid] in (
                "Invasive Diastolic Arterial Blood Pressure",
                "Non-Invasive Diastolic Arterial Blood Pressure",
            ):
                var_id = "diastolic_arterial_bp_computed"
            if var_bindings["dynamic"][tup.variableid] in (
                "Invasive Mean Arterial Blood Pressure",
                "Non-Invasive Mean Arterial Blood Pressure",
            ):
                var_id = "mean_arterial_bp_computed"
            if var_bindings["dynamic"][tup.variableid] in (
                "Invasive Systolic Arterial Blood Pressure",
                "Non-Invasive Systolic Arterial Blood Pressure",
            ):
                var_id = "systolic_arterial_bp_computed"
            # check bp variables priority (use invasive, if not available use non-invasive)
            if tup.variableid not in ("100", "110", "120"):
                # non-invasive bp, check if there's some bp value already present
                if tmp_dict[row_key][var_bindings["dynamic"][var_id]] is not None:
                    # bp value already present, skip since it's probably an invasive one
                    continue
            tmp_dict[row_key][var_bindings["dynamic"][var_id]] = tup.value

    df_dynamic_sample = pd.DataFrame(
        data=tmp_dict.values(),
        columns=dynamic_columns,
    )
    df_dynamic_sample["Patient ID"] = pid
    df_dynamic_sample.set_index(
        pd.Index(tmp_dict.keys(), name="Minutes of Stay"),
        drop=True,
        inplace=True,
    )
    df_dynamic_sample.to_csv(f"./results/{tup.patientid}/{tup.patientid}_dynamic.csv")


# Check Variables' Distributions


In [ ]:
patientids = list()
with open("./results/patients_18+_1d+_2+lactate_measurements_in_first2d.txt", "r") as f:
    patientids.extend(int(x) for x in f.readlines())

var_bindings = dict()
with open("./var_id_var_name_bindings.json", "r") as f:
    var_bindings = json.load(f)

os.makedirs("./results/distribution_plots", exist_ok=True)
# avoid duplicated columns
dynamic_columns = list(dict.fromkeys(var_bindings["dynamic"].values()).keys())
pids_checks = "', '".join([str(x) for x in patientids])
# skip patientid, datetime, entertime
for var in dynamic_columns[3:]:
    # remove invalid characters from filename
    filename = var
    for c in ("\\", "/", ":", "*", "?", '"', "<", ">", "|"):
        filename = filename.replace(c, "_")
    # get all HiRID var_ids connected to final variable name
    var_X_ids = [k for k, v in var_bindings["dynamic"].items() if v == var]
    print(var, var_X_ids)
    var_ids_checks = "', '".join(var_X_ids)
    conn = sqlite3.connect(DB_PATH)
    df_var_raw = pd.read_sql(
        f"SELECT value FROM raw_data WHERE (patientid IN ('{pids_checks}')) AND (variableid IN ('{var_ids_checks}'))",
        conn,
    )
    df_var_raw.to_csv(f"./results/distribution_plots/{filename}.csv")
    conn.close()
    df_var_raw = pd.read_csv(
        f"./results/distribution_plots/{filename}.csv", index_col=0
    )


# Run plots.R to get distribution plots
